In [1]:
import pandas as pd
import numpy as np


In [2]:
!pip install imbalance-xgboost


     |████████████████████████████████| 173.5 MB 42 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb


In [3]:
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import gensim

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df=pd.read_csv('/content/drive/MyDrive/Isarcasm/Dataset/train.En.csv')

In [7]:
df=df[['tweet','sarcastic']]
df.dropna(inplace=True)

In [ ]:
# train, validate, test = \
#               np.split(df.sample(frac=1, random_state=42), 
#                        [int(.6*len(df)), int(.8*len(df))])

In [8]:
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(df['tweet'])

In [10]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['tweet'])

In [8]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
tokenized_tweet = df['tweet'].apply(word_tokenize) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model_w2v.train(tokenized_tweet, total_examples= len(df['tweet']), epochs=20)

(1000350, 1528100)

In [112]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            # print(model_w2v[word].shape)
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [113]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    # print(i)
    wordvec_arrays[i,:] = word_vector(tokenized_tweet.iloc[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [15]:
from tqdm import tqdm 
tqdm.pandas(desc="progress-bar") 
from gensim.models.doc2vec import LabeledSentence
def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ["tweet_" + str(i)]))
    return output

labeled_tweets = add_label(tokenized_tweet) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  import sys


In [16]:
%%time 
model_d2v = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model
                                  dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                                  vector_size=200, # no. of desired features
                                  window=5, # width of the context window                                  
                                  negative=7, # if > 0 then negative sampling will be used
                                  min_count=5, # Ignores all words with total frequency lower than 5.                                  
                                  workers=32, # no. of cores                                  
                                  alpha=0.1, # learning rate                                  
                                  seed = 23, # for reproducibility
                                 ) 

model_d2v.build_vocab([i for i in tqdm(labeled_tweets)])

model_d2v.train(labeled_tweets, total_examples= len(df['tweet']), epochs=15)
docvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(df)):
    docvec_arrays[i,:] = model_d2v.docvecs[i].reshape((1,200))    

docvec_df = pd.DataFrame(docvec_arrays) 

100%|██████████| 3467/3467 [00:00<00:00, 1342347.64it/s]


CPU times: user 11.1 s, sys: 2.64 s, total: 13.7 s
Wall time: 10 s


In [17]:
xtrain_bow, xtest_bow, ytrain_bow, yvtest_bow = train_test_split(bow, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_tfidf, xtest_tfidf, ytrain_tfidf, yvtest_tfidf = train_test_split(tfidf, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_wordvec, xtest_wordvec, ytrain_wordvec, yvtest_wordvec = train_test_split(wordvec_df, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_docvec, xtest_docvec, ytrain_docvec, yvtest_docvec = train_test_split(docvec_df, df['sarcastic'], random_state=42, test_size=0.3)

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [9]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = RandomForestClassifier(n_estimators=400, random_state=11)
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =XGBClassifier(max_depth=6, n_estimators=1000)
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=LinearSVC(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [22]:
xtrain_bow, xtest_bow, ytrain_bow, yvtest_bow = train_test_split(bow, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_tfidf, xtest_tfidf, ytrain_tfidf, yvtest_tfidf = train_test_split(tfidf, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_wordvec, xtest_wordvec, ytrain_wordvec, yvtest_wordvec = train_test_split(wordvec_df, df['sarcastic'], random_state=42, test_size=0.3)
xtrain_docvec, xtest_docvec, ytrain_docvec, yvtest_docvec = train_test_split(docvec_df, df['sarcastic'], random_state=42, test_size=0.3)

In [23]:
linear_svm_grid(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.605
Precision: 0.635
Recall: 0.605
F_score: 0.617

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       754
           1       0.33      0.41      0.36       287

    accuracy                           0.61      1041
   macro avg       0.54      0.55      0.54      1041
weighted avg       0.63      0.61      0.62      1041



In [24]:
linear_svm_grid(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.605
Precision: 0.635
Recall: 0.605
F_score: 0.617

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       754
           1       0.33      0.41      0.36       287

    accuracy                           0.61      1041
   macro avg       0.54      0.55      0.54      1041
weighted avg       0.63      0.61      0.62      1041



In [25]:
linear_svm_grid(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.611
Precision: 0.641
Recall: 0.611
F_score: 0.623

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       754
           1       0.34      0.43      0.38       287

    accuracy                           0.61      1041
   macro avg       0.55      0.55      0.55      1041
weighted avg       0.64      0.61      0.62      1041



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [26]:
linear_svm_grid(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.660
Precision: 0.633
Recall: 0.660
F_score: 0.644

              precision    recall  f1-score   support

           0       0.74      0.81      0.78       754
           1       0.35      0.26      0.30       287

    accuracy                           0.66      1041
   macro avg       0.54      0.54      0.54      1041
weighted avg       0.63      0.66      0.64      1041



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [27]:
nonlinear_svm_grid(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.713
Precision: 0.643
Recall: 0.713
F_score: 0.640

              precision    recall  f1-score   support

           0       0.73      0.95      0.83       754
           1       0.41      0.09      0.15       287

    accuracy                           0.71      1041
   macro avg       0.57      0.52      0.49      1041
weighted avg       0.64      0.71      0.64      1041



In [28]:
nonlinear_svm_grid(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.672
Precision: 0.645
Recall: 0.672
F_score: 0.656

              precision    recall  f1-score   support

           0       0.75      0.82      0.78       754
           1       0.37      0.28      0.32       287

    accuracy                           0.67      1041
   macro avg       0.56      0.55      0.55      1041
weighted avg       0.65      0.67      0.66      1041



In [29]:
nonlinear_svm_grid(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.611
Precision: 0.635
Recall: 0.611
F_score: 0.621

              precision    recall  f1-score   support

           0       0.75      0.69      0.72       754
           1       0.33      0.40      0.36       287

    accuracy                           0.61      1041
   macro avg       0.54      0.54      0.54      1041
weighted avg       0.63      0.61      0.62      1041



In [30]:
nonlinear_svm_grid(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.684
Precision: 0.629
Recall: 0.684
F_score: 0.644

              precision    recall  f1-score   support

           0       0.74      0.88      0.80       754
           1       0.35      0.17      0.23       287

    accuracy                           0.68      1041
   macro avg       0.54      0.53      0.52      1041
weighted avg       0.63      0.68      0.64      1041



In [31]:
logistic_regression_grid(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.613
Precision: 0.638
Recall: 0.613
F_score: 0.624

              precision    recall  f1-score   support

           0       0.75      0.69      0.72       754
           1       0.33      0.41      0.37       287

    accuracy                           0.61      1041
   macro avg       0.54      0.55      0.54      1041
weighted avg       0.64      0.61      0.62      1041



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [32]:
logistic_regression_grid(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.600
Precision: 0.626
Recall: 0.600
F_score: 0.611

              precision    recall  f1-score   support

           0       0.74      0.68      0.71       754
           1       0.32      0.38      0.35       287

    accuracy                           0.60      1041
   macro avg       0.53      0.53      0.53      1041
weighted avg       0.63      0.60      0.61      1041



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [33]:
logistic_regression_grid(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.575
Precision: 0.648
Recall: 0.575
F_score: 0.597

              precision    recall  f1-score   support

           0       0.77      0.59      0.67       754
           1       0.33      0.53      0.41       287

    accuracy                           0.58      1041
   macro avg       0.55      0.56      0.54      1041
weighted avg       0.65      0.58      0.60      1041



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [34]:
logistic_regression_grid(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.592
Precision: 0.642
Recall: 0.592
F_score: 0.610

              precision    recall  f1-score   support

           0       0.76      0.64      0.69       754
           1       0.33      0.47      0.39       287

    accuracy                           0.59      1041
   macro avg       0.55      0.56      0.54      1041
weighted avg       0.64      0.59      0.61      1041



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
linear_svm(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


          Linear SVM

Accuracy: 0.617
Precision: 0.644
Recall: 0.617
F_score: 0.628

              precision    recall  f1-score   support

           0       0.76      0.69      0.72       754
           1       0.34      0.42      0.38       287

    accuracy                           0.62      1041
   macro avg       0.55      0.56      0.55      1041
weighted avg       0.64      0.62      0.63      1041



In [36]:
linear_svm(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


          Linear SVM

Accuracy: 0.601
Precision: 0.643
Recall: 0.601
F_score: 0.617

              precision    recall  f1-score   support

           0       0.76      0.66      0.70       754
           1       0.34      0.46      0.39       287

    accuracy                           0.60      1041
   macro avg       0.55      0.56      0.55      1041
weighted avg       0.64      0.60      0.62      1041



In [37]:
linear_svm(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


          Linear SVM

Accuracy: 0.572
Precision: 0.653
Recall: 0.572
F_score: 0.594

              precision    recall  f1-score   support

           0       0.77      0.58      0.66       754
           1       0.33      0.56      0.42       287

    accuracy                           0.57      1041
   macro avg       0.55      0.57      0.54      1041
weighted avg       0.65      0.57      0.59      1041



In [38]:
linear_svm(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


          Linear SVM

Accuracy: 0.553
Precision: 0.631
Recall: 0.553
F_score: 0.576

              precision    recall  f1-score   support

           0       0.75      0.57      0.65       754
           1       0.31      0.51      0.39       287

    accuracy                           0.55      1041
   macro avg       0.53      0.54      0.52      1041
weighted avg       0.63      0.55      0.58      1041



In [39]:
logistic_regression(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.614
Precision: 0.645
Recall: 0.614
F_score: 0.626

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       754
           1       0.34      0.44      0.38       287

    accuracy                           0.61      1041
   macro avg       0.55      0.56      0.55      1041
weighted avg       0.65      0.61      0.63      1041



In [40]:
logistic_regression(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.602
Precision: 0.649
Recall: 0.602
F_score: 0.619

              precision    recall  f1-score   support

           0       0.77      0.65      0.70       754
           1       0.34      0.48      0.40       287

    accuracy                           0.60      1041
   macro avg       0.55      0.56      0.55      1041
weighted avg       0.65      0.60      0.62      1041



In [41]:
logistic_regression(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.546
Precision: 0.640
Recall: 0.546
F_score: 0.570

              precision    recall  f1-score   support

           0       0.76      0.54      0.63       754
           1       0.32      0.56      0.41       287

    accuracy                           0.55      1041
   macro avg       0.54      0.55      0.52      1041
weighted avg       0.64      0.55      0.57      1041



In [42]:
logistic_regression(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.568
Precision: 0.642
Recall: 0.568
F_score: 0.590

              precision    recall  f1-score   support

           0       0.76      0.58      0.66       754
           1       0.32      0.52      0.40       287

    accuracy                           0.57      1041
   macro avg       0.54      0.55      0.53      1041
weighted avg       0.64      0.57      0.59      1041



In [69]:
random_forest(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf,class_ratio='balanced')


       Random Forest

Accuracy: 0.698
Precision: 0.623
Recall: 0.698
F_score: 0.634

              precision    recall  f1-score   support

           0       0.73      0.93      0.82       754
           1       0.34      0.10      0.16       287

    accuracy                           0.70      1041
   macro avg       0.54      0.51      0.49      1041
weighted avg       0.62      0.70      0.63      1041



In [44]:
random_forest(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow,class_ratio='balanced')


       Random Forest

Accuracy: 0.650
Precision: 0.619
Recall: 0.650
F_score: 0.631

              precision    recall  f1-score   support

           0       0.73      0.81      0.77       754
           1       0.32      0.23      0.27       287

    accuracy                           0.65      1041
   macro avg       0.53      0.52      0.52      1041
weighted avg       0.62      0.65      0.63      1041



In [45]:
random_forest(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec,class_ratio='balanced')


       Random Forest

Accuracy: 0.722
Precision: 0.629
Recall: 0.722
F_score: 0.613

              precision    recall  f1-score   support

           0       0.73      0.99      0.84       754
           1       0.38      0.01      0.02       287

    accuracy                           0.72      1041
   macro avg       0.55      0.50      0.43      1041
weighted avg       0.63      0.72      0.61      1041



In [46]:
random_forest(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec,class_ratio='balanced')


       Random Forest

Accuracy: 0.725
Precision: 0.709
Recall: 0.725
F_score: 0.613

              precision    recall  f1-score   support

           0       0.73      1.00      0.84       754
           1       0.67      0.01      0.01       287

    accuracy                           0.73      1041
   macro avg       0.70      0.50      0.43      1041
weighted avg       0.71      0.73      0.61      1041



In [47]:
xg_boost(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


             XGBoost

[13:29:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.692
Precision: 0.650
Recall: 0.692
F_score: 0.661

              precision    recall  f1-score   support

           0       0.75      0.87      0.80       754
           1       0.40      0.23      0.29       287

    accuracy                           0.69      1041
   macro avg       0.57      0.55      0.55      1041
weighted avg       0.65      0.69      0.66      1041



In [70]:
xg_boost(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


             XGBoost

[13:59:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.680
Precision: 0.637
Recall: 0.680
F_score: 0.650

              precision    recall  f1-score   support

           0       0.74      0.86      0.80       754
           1       0.36      0.21      0.27       287

    accuracy                           0.68      1041
   macro avg       0.55      0.54      0.53      1041
weighted avg       0.64      0.68      0.65      1041



In [49]:
xg_boost(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


             XGBoost

[13:29:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.705
Precision: 0.615
Recall: 0.705
F_score: 0.625

              precision    recall  f1-score   support

           0       0.73      0.95      0.82       754
           1       0.32      0.06      0.10       287

    accuracy                           0.71      1041
   macro avg       0.52      0.51      0.46      1041
weighted avg       0.62      0.71      0.63      1041



In [50]:
xg_boost(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)


             XGBoost

[13:30:38] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.699
Precision: 0.603
Recall: 0.699
F_score: 0.621

              precision    recall  f1-score   support

           0       0.72      0.94      0.82       754
           1       0.28      0.06      0.10       287

    accuracy                           0.70      1041
   macro avg       0.50      0.50      0.46      1041
weighted avg       0.60      0.70      0.62      1041



In [58]:
xg_boost_focal_loss(xtrain_tfidf,ytrain_tfidf.to_numpy(),xtest_tfidf,yvtest_tfidf.to_numpy())


       XGBoost Focal

Accuracy: 0.716
Precision: 0.634
Recall: 0.716
F_score: 0.628

              precision    recall  f1-score   support

           0       0.73      0.97      0.83       754
           1       0.38      0.05      0.09       287

    accuracy                           0.72      1041
   macro avg       0.56      0.51      0.46      1041
weighted avg       0.63      0.72      0.63      1041



In [59]:
xg_boost_focal_loss(xtrain_bow,ytrain_bow.to_numpy(),xtest_bow,yvtest_bow.to_numpy())


       XGBoost Focal

Accuracy: 0.718
Precision: 0.631
Recall: 0.718
F_score: 0.623

              precision    recall  f1-score   support

           0       0.73      0.98      0.83       754
           1       0.38      0.04      0.07       287

    accuracy                           0.72      1041
   macro avg       0.55      0.51      0.45      1041
weighted avg       0.63      0.72      0.62      1041



In [62]:
xg_boost_weighted_loss(xtrain_bow,ytrain_bow.to_numpy(),xtest_bow,yvtest_bow.to_numpy())


    XGBoost Weighted

Accuracy: 0.720
Precision: 0.653
Recall: 0.720
F_score: 0.637

              precision    recall  f1-score   support

           0       0.73      0.97      0.83       754
           1       0.44      0.07      0.12       287

    accuracy                           0.72      1041
   macro avg       0.59      0.52      0.48      1041
weighted avg       0.65      0.72      0.64      1041



In [63]:
xg_boost_weighted_loss(xtrain_tfidf,ytrain_tfidf.to_numpy(),xtest_tfidf,yvtest_tfidf.to_numpy())


    XGBoost Weighted

Accuracy: 0.710
Precision: 0.623
Recall: 0.710
F_score: 0.627

              precision    recall  f1-score   support

           0       0.73      0.96      0.83       754
           1       0.35      0.06      0.10       287

    accuracy                           0.71      1041
   macro avg       0.54      0.51      0.46      1041
weighted avg       0.62      0.71      0.63      1041



In [71]:
ensemble_stacked(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


Ensemble Stacked Classifiers

[14:00:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:00:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:00:41] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:00:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:00:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [73]:
ensemble_stacked(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


Ensemble Stacked Classifiers

[14:02:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:02:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:02:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:03:04] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:03:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [74]:
ensemble_stacked(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:03:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:04:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:06:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
ensemble_stacked(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Ensemble Stacked Classifiers

[14:07:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:07:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:08:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:08:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:09:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
voting_classifiers(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


   Voting Classifier

[14:10:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.681
Precision: 0.646
Recall: 0.681
F_score: 0.657

              precision    recall  f1-score   support

           0       0.75      0.85      0.79       754
           1       0.38      0.25      0.30       287

    accuracy                           0.68      1041
   macro avg       0.56      0.55      0.55      1041
weighted avg       0.65      0.68      0.66      1041



In [77]:
voting_classifiers(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


   Voting Classifier

[14:10:55] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accuracy: 0.670
Precision: 0.630
Recall: 0.670
F_score: 0.644

              precision    recall  f1-score   support

           0       0.74      0.84      0.79       754
           1       0.34      0.22      0.27       287

    accuracy                           0.67      1041
   macro avg       0.54      0.53      0.53      1041
weighted avg       0.63      0.67      0.64      1041



In [78]:
voting_classifiers(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:11:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.699
Precision: 0.607
Recall: 0.699
F_score: 0.623

              precision    recall  f1-score   support

           0       0.73      0.94      0.82       754
           1       0.30      0.07      0.11       287

    accuracy                           0.70      1041
   macro avg       0.51      0.50      0.46      1041
weighted avg       0.61      0.70      0.62      1041



In [79]:
voting_classifiers(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:11:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.688
Precision: 0.590
Recall: 0.688
F_score: 0.615

              precision    recall  f1-score   support

           0       0.72      0.93      0.81       754
           1       0.24      0.06      0.10       287

    accuracy                           0.69      1041
   macro avg       0.48      0.49      0.46      1041
weighted avg       0.59      0.69      0.62      1041



In [83]:
Bagging_Classifier_LR(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


Bagging Calssifier LR

Accuracy: 0.606
Precision: 0.635
Recall: 0.606
F_score: 0.618

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       754
           1       0.33      0.41      0.36       287

    accuracy                           0.61      1041
   macro avg       0.54      0.54      0.54      1041
weighted avg       0.63      0.61      0.62      1041



In [84]:
Bagging_Classifier_LR(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


Bagging Calssifier LR

Accuracy: 0.452
Precision: 0.662
Recall: 0.452
F_score: 0.458

              precision    recall  f1-score   support

           0       0.80      0.33      0.46       754
           1       0.31      0.78      0.44       287

    accuracy                           0.45      1041
   macro avg       0.55      0.55      0.45      1041
weighted avg       0.66      0.45      0.46      1041



In [85]:
Bagging_Classifier_LR(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


Bagging Calssifier LR

Accuracy: 0.501
Precision: 0.651
Recall: 0.501
F_score: 0.522

              precision    recall  f1-score   support

           0       0.78      0.44      0.56       754
           1       0.31      0.68      0.43       287

    accuracy                           0.50      1041
   macro avg       0.55      0.56      0.49      1041
weighted avg       0.65      0.50      0.52      1041



In [86]:
Bagging_Classifier_LR(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)


Bagging Calssifier LR

Accuracy: 0.558
Precision: 0.649
Recall: 0.558
F_score: 0.582

              precision    recall  f1-score   support

           0       0.77      0.55      0.65       754
           1       0.33      0.57      0.41       287

    accuracy                           0.56      1041
   macro avg       0.55      0.56      0.53      1041
weighted avg       0.65      0.56      0.58      1041



In [87]:
Bagging_Classifier_SVM(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


Bagging Calssifier SVM

Accuracy: 0.635
Precision: 0.630
Recall: 0.635
F_score: 0.633

              precision    recall  f1-score   support

           0       0.74      0.76      0.75       754
           1       0.33      0.32      0.32       287

    accuracy                           0.63      1041
   macro avg       0.54      0.54      0.54      1041
weighted avg       0.63      0.63      0.63      1041



In [88]:
Bagging_Classifier_SVM(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


Bagging Calssifier SVM

Accuracy: 0.626
Precision: 0.631
Recall: 0.626
F_score: 0.628

              precision    recall  f1-score   support

           0       0.75      0.73      0.74       754
           1       0.33      0.34      0.34       287

    accuracy                           0.63      1041
   macro avg       0.54      0.54      0.54      1041
weighted avg       0.63      0.63      0.63      1041



In [89]:
Bagging_Classifier_SVM(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


Bagging Calssifier SVM

Accuracy: 0.597
Precision: 0.657
Recall: 0.597
F_score: 0.616

              precision    recall  f1-score   support

           0       0.78      0.62      0.69       754
           1       0.35      0.53      0.42       287

    accuracy                           0.60      1041
   macro avg       0.56      0.57      0.55      1041
weighted avg       0.66      0.60      0.62      1041



In [90]:
Bagging_Classifier_SVM(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)


Bagging Calssifier SVM

Accuracy: 0.585
Precision: 0.644
Recall: 0.585
F_score: 0.605

              precision    recall  f1-score   support

           0       0.76      0.62      0.68       754
           1       0.33      0.49      0.40       287

    accuracy                           0.59      1041
   macro avg       0.55      0.56      0.54      1041
weighted avg       0.64      0.59      0.60      1041



In [91]:
gradient_boosting(xtrain_bow,ytrain_bow,xtest_bow,yvtest_bow)


   Gradient Boosting

Accuracy: 0.712
Precision: 0.629
Recall: 0.712
F_score: 0.629

              precision    recall  f1-score   support

           0       0.73      0.96      0.83       754
           1       0.37      0.06      0.11       287

    accuracy                           0.71      1041
   macro avg       0.55      0.51      0.47      1041
weighted avg       0.63      0.71      0.63      1041



In [92]:
gradient_boosting(xtrain_tfidf,ytrain_tfidf,xtest_tfidf,yvtest_tfidf)


   Gradient Boosting

Accuracy: 0.709
Precision: 0.616
Recall: 0.709
F_score: 0.624

              precision    recall  f1-score   support

           0       0.73      0.96      0.83       754
           1       0.33      0.05      0.09       287

    accuracy                           0.71      1041
   macro avg       0.53      0.51      0.46      1041
weighted avg       0.62      0.71      0.62      1041



In [93]:
gradient_boosting(xtrain_wordvec,ytrain_wordvec,xtest_wordvec,yvtest_wordvec)


   Gradient Boosting

Accuracy: 0.676
Precision: 0.629
Recall: 0.676
F_score: 0.644

              precision    recall  f1-score   support

           0       0.74      0.86      0.79       754
           1       0.35      0.20      0.25       287

    accuracy                           0.68      1041
   macro avg       0.54      0.53      0.52      1041
weighted avg       0.63      0.68      0.64      1041



In [94]:
gradient_boosting(xtrain_docvec,ytrain_docvec,xtest_docvec,yvtest_docvec)


   Gradient Boosting

Accuracy: 0.640
Precision: 0.604
Recall: 0.640
F_score: 0.618

              precision    recall  f1-score   support

           0       0.73      0.81      0.76       754
           1       0.28      0.20      0.23       287

    accuracy                           0.64      1041
   macro avg       0.50      0.50      0.50      1041
weighted avg       0.60      0.64      0.62      1041



In [95]:
!pip install transformers


     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [99]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
           output_hidden_states = True,)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [118]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [117]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # print(token_embeddings.shape)
    # token_vecs_cat = []
    # for token in token_embeddings:
    #   cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [10]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [11]:
train=train.append(validate, ignore_index=True)

In [120]:
target_word_embeddings = []
wordvec_arrays = np.zeros((len(train['tweet']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in train['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [124]:
target_word_embeddings = []
wordvec_arrays = np.zeros((len(test['tweet']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in test['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [122]:
wordvec_df_new = pd.DataFrame(wordvec_arrays)

In [125]:
wordvec_df_test= pd.DataFrame(wordvec_arrays)

In [126]:
gradient_boosting(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Gradient Boosting

Accuracy: 0.692
Precision: 0.662
Recall: 0.692
F_score: 0.675

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       532
           1       0.28      0.21      0.24       162

    accuracy                           0.69       694
   macro avg       0.53      0.52      0.52       694
weighted avg       0.66      0.69      0.67       694



In [127]:
Bagging_Classifier_SVM(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier SVM

Accuracy: 0.700
Precision: 0.721
Recall: 0.700
F_score: 0.709

              precision    recall  f1-score   support

           0       0.82      0.77      0.80       532
           1       0.38      0.46      0.42       162

    accuracy                           0.70       694
   macro avg       0.60      0.62      0.61       694
weighted avg       0.72      0.70      0.71       694



In [128]:
Bagging_Classifier_LR(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier LR

Accuracy: 0.656
Precision: 0.723
Recall: 0.656
F_score: 0.678

              precision    recall  f1-score   support

           0       0.84      0.68      0.75       532
           1       0.35      0.56      0.43       162

    accuracy                           0.66       694
   macro avg       0.59      0.62      0.59       694
weighted avg       0.72      0.66      0.68       694



In [129]:
voting_classifiers(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.761
Precision: 0.702
Recall: 0.761
F_score: 0.699

              precision    recall  f1-score   support

           0       0.78      0.96      0.86       532
           1       0.45      0.10      0.17       162

    accuracy                           0.76       694
   macro avg       0.61      0.53      0.52       694
weighted avg       0.70      0.76      0.70       694



In [130]:
ensemble_stacked(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:12:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:15:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:17:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:19:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:21:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [131]:
xg_boost(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


             XGBoost



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:24:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.762
Precision: 0.708
Recall: 0.762
F_score: 0.705

              precision    recall  f1-score   support

           0       0.78      0.96      0.86       532
           1       0.47      0.12      0.20       162

    accuracy                           0.76       694
   macro avg       0.62      0.54      0.53       694
weighted avg       0.71      0.76      0.71       694



In [132]:
random_forest(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Random Forest

Accuracy: 0.764
Precision: 0.587
Recall: 0.764
F_score: 0.664

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.76       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.76      0.66       694



In [133]:
logistic_regression(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.633
Precision: 0.718
Recall: 0.633
F_score: 0.659

              precision    recall  f1-score   support

           0       0.84      0.65      0.73       532
           1       0.33      0.58      0.42       162

    accuracy                           0.63       694
   macro avg       0.58      0.61      0.58       694
weighted avg       0.72      0.63      0.66       694



In [134]:
linear_svm(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.674
Precision: 0.733
Recall: 0.674
F_score: 0.694

              precision    recall  f1-score   support

           0       0.84      0.71      0.77       532
           1       0.37      0.57      0.45       162

    accuracy                           0.67       694
   macro avg       0.61      0.64      0.61       694
weighted avg       0.73      0.67      0.69       694



In [135]:
logistic_regression_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.671
Precision: 0.719
Recall: 0.671
F_score: 0.689

              precision    recall  f1-score   support

           0       0.83      0.72      0.77       532
           1       0.36      0.51      0.42       162

    accuracy                           0.67       694
   macro avg       0.59      0.62      0.60       694
weighted avg       0.72      0.67      0.69       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [136]:
nonlinear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.765
Precision: 0.587
Recall: 0.765
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.66       694



In [137]:
linear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.661
Precision: 0.712
Recall: 0.661
F_score: 0.680

              precision    recall  f1-score   support

           0       0.82      0.71      0.76       532
           1       0.34      0.50      0.41       162

    accuracy                           0.66       694
   macro avg       0.58      0.61      0.59       694
weighted avg       0.71      0.66      0.68       694



In [138]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 2.8 MB/s 
     |████████████████████████████████| 1.2 MB 8.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=1453b7860ffb35dd4e72842e5ea7ce2517691749e2bb837ac6f8190541b46d39
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [139]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [145]:
target_sent_embeddings_train = []
for text in train['tweet'].values:
  sentence_embeddings = model.encode(text)
  # print(sentence_embeddings)
  target_sent_embeddings_train.append(sentence_embeddings)

sent2_vec_df_new_train = pd.DataFrame(target_sent_embeddings_train)

In [146]:
target_sent_embeddings_test = []
for text in test['tweet'].values:
  sentence_embeddings = model.encode(text)
  target_sent_embeddings_test.append(sentence_embeddings)

sent2_vec_df_new_test = pd.DataFrame(target_sent_embeddings_test)

In [148]:
gradient_boosting(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


   Gradient Boosting

Accuracy: 0.669
Precision: 0.639
Recall: 0.669
F_score: 0.652

              precision    recall  f1-score   support

           0       0.76      0.82      0.79       532
           1       0.23      0.17      0.20       162

    accuracy                           0.67       694
   macro avg       0.50      0.50      0.49       694
weighted avg       0.64      0.67      0.65       694



In [149]:
Bagging_Classifier_SVM(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


Bagging Calssifier SVM

Accuracy: 0.621
Precision: 0.703
Recall: 0.621
F_score: 0.648

              precision    recall  f1-score   support

           0       0.82      0.65      0.72       532
           1       0.32      0.54      0.40       162

    accuracy                           0.62       694
   macro avg       0.57      0.59      0.56       694
weighted avg       0.70      0.62      0.65       694



In [150]:
Bagging_Classifier_LR(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


Bagging Calssifier LR

Accuracy: 0.640
Precision: 0.707
Recall: 0.640
F_score: 0.663

              precision    recall  f1-score   support

           0       0.82      0.68      0.74       532
           1       0.33      0.52      0.40       162

    accuracy                           0.64       694
   macro avg       0.58      0.60      0.57       694
weighted avg       0.71      0.64      0.66       694



In [151]:
voting_classifiers(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:48:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.758
Precision: 0.694
Recall: 0.758
F_score: 0.696

              precision    recall  f1-score   support

           0       0.78      0.96      0.86       532
           1       0.42      0.10      0.16       162

    accuracy                           0.76       694
   macro avg       0.60      0.53      0.51       694
weighted avg       0.69      0.76      0.70       694



In [152]:
ensemble_stacked(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:50:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:51:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:52:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:53:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:54:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [153]:
xg_boost(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'])


             XGBoost



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:56:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.768
Precision: 0.718
Recall: 0.768
F_score: 0.702

              precision    recall  f1-score   support

           0       0.78      0.97      0.87       532
           1       0.52      0.10      0.17       162

    accuracy                           0.77       694
   macro avg       0.65      0.54      0.52       694
weighted avg       0.72      0.77      0.70       694



In [154]:
random_forest(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


       Random Forest

Accuracy: 0.765
Precision: 0.587
Recall: 0.765
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.66       694



In [155]:
logistic_regression(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.578
Precision: 0.703
Recall: 0.578
F_score: 0.610

              precision    recall  f1-score   support

           0       0.83      0.57      0.67       532
           1       0.30      0.60      0.40       162

    accuracy                           0.58       694
   macro avg       0.56      0.59      0.54       694
weighted avg       0.70      0.58      0.61       694



In [156]:
linear_svm(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.591
Precision: 0.702
Recall: 0.591
F_score: 0.622

              precision    recall  f1-score   support

           0       0.82      0.59      0.69       532
           1       0.30      0.58      0.40       162

    accuracy                           0.59       694
   macro avg       0.56      0.59      0.54       694
weighted avg       0.70      0.59      0.62       694



In [157]:
logistic_regression_grid(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.608
Precision: 0.701
Recall: 0.608
F_score: 0.637

              precision    recall  f1-score   support

           0       0.82      0.63      0.71       532
           1       0.31      0.55      0.40       162

    accuracy                           0.61       694
   macro avg       0.56      0.59      0.55       694
weighted avg       0.70      0.61      0.64       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [158]:
nonlinear_svm_grid(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.233
Precision: 0.054
Recall: 0.233
F_score: 0.088

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       532
           1       0.23      1.00      0.38       162

    accuracy                           0.23       694
   macro avg       0.12      0.50      0.19       694
weighted avg       0.05      0.23      0.09       694



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [159]:
linear_svm_grid(sent2_vec_df_new_train,train['sarcastic'],sent2_vec_df_new_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.602
Precision: 0.699
Recall: 0.602
F_score: 0.632

              precision    recall  f1-score   support

           0       0.82      0.62      0.70       532
           1       0.30      0.55      0.39       162

    accuracy                           0.60       694
   macro avg       0.56      0.58      0.55       694
weighted avg       0.70      0.60      0.63       694



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [163]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertModel.from_pretrained('distilbert-base-uncased',
           output_hidden_states = True,)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.3.ffn.lin2.weight', 'distilbert.transformer.layer.5.attention.out_lin.weight', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.1.ffn.lin2.bias', 'distilbert.transformer.layer.5.output_layer_norm.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.2.attention.v_lin.weight', 'vocab_layer_norm.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.1.ffn.lin1.weight', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.4.attention.v_lin.bias', 'distilbert.transformer.layer.2.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.k_lin.bias', 'di

In [164]:
wordvec_arrays_train = np.zeros((len(train['tweet']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in train['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays_train[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [165]:
wordvec_arrays_test = np.zeros((len(test['tweet']), 768)) 
i=0
# for i in range(len(tokenized_tweet)):
#     # print(i)
    


for text in test['tweet'].values:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    def word_vector(tokens, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec
    wordvec_arrays_test[i,:] = word_vector(list_token_embeddings, 768)
    i=i+1
    # print(len(list_token_embeddings))
    # target_word_embeddings.append(np.array(list_token_embeddings))
    
    # # Find the position 'bank' in list of tokens
    # word_index = tokenized_text.index('bank')
    # # Get the embedding for bank
    # word_embedding = list_token_embeddings[word_index]

    # target_word_embeddings.append(word_embedding)

In [166]:
wordvec_df_new = pd.DataFrame(wordvec_arrays_train)

In [167]:
wordvec_df_test= pd.DataFrame(wordvec_arrays_test)

In [168]:
gradient_boosting(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Gradient Boosting

Accuracy: 0.657
Precision: 0.642
Recall: 0.657
F_score: 0.649

              precision    recall  f1-score   support

           0       0.77      0.80      0.78       532
           1       0.23      0.20      0.22       162

    accuracy                           0.66       694
   macro avg       0.50      0.50      0.50       694
weighted avg       0.64      0.66      0.65       694



In [169]:
Bagging_Classifier_SVM(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier SVM

Accuracy: 0.634
Precision: 0.694
Recall: 0.634
F_score: 0.656

              precision    recall  f1-score   support

           0       0.81      0.68      0.74       532
           1       0.31      0.48      0.38       162

    accuracy                           0.63       694
   macro avg       0.56      0.58      0.56       694
weighted avg       0.69      0.63      0.66       694



In [170]:
Bagging_Classifier_LR(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier LR



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.592
Precision: 0.687
Recall: 0.592
F_score: 0.622

              precision    recall  f1-score   support

           0       0.81      0.61      0.70       532
           1       0.29      0.52      0.37       162

    accuracy                           0.59       694
   macro avg       0.55      0.57      0.54       694
weighted avg       0.69      0.59      0.62       694



In [171]:
voting_classifiers(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:22:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.748
Precision: 0.664
Recall: 0.748
F_score: 0.680

              precision    recall  f1-score   support

           0       0.77      0.95      0.85       532
           1       0.31      0.07      0.11       162

    accuracy                           0.75       694
   macro avg       0.54      0.51      0.48       694
weighted avg       0.66      0.75      0.68       694



In [172]:
ensemble_stacked(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:25:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[16:28:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:30:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:34:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [173]:
xg_boost(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



             XGBoost

[16:38:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.756
Precision: 0.690
Recall: 0.756
F_score: 0.693

              precision    recall  f1-score   support

           0       0.78      0.96      0.86       532
           1       0.41      0.09      0.15       162

    accuracy                           0.76       694
   macro avg       0.59      0.53      0.50       694
weighted avg       0.69      0.76      0.69       694



In [174]:
random_forest(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Random Forest

Accuracy: 0.765
Precision: 0.587
Recall: 0.765
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.66       694



In [175]:
logistic_regression(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.578
Precision: 0.687
Recall: 0.578
F_score: 0.610

              precision    recall  f1-score   support

           0       0.81      0.59      0.68       532
           1       0.29      0.54      0.38       162

    accuracy                           0.58       694
   macro avg       0.55      0.57      0.53       694
weighted avg       0.69      0.58      0.61       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [176]:
linear_svm(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.592
Precision: 0.694
Recall: 0.592
F_score: 0.623

              precision    recall  f1-score   support

           0       0.82      0.61      0.69       532
           1       0.30      0.55      0.39       162

    accuracy                           0.59       694
   macro avg       0.56      0.58      0.54       694
weighted avg       0.69      0.59      0.62       694



In [177]:
logistic_regression_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.598
Precision: 0.680
Recall: 0.598
F_score: 0.626

              precision    recall  f1-score   support

           0       0.80      0.63      0.71       532
           1       0.29      0.48      0.36       162

    accuracy                           0.60       694
   macro avg       0.54      0.56      0.53       694
weighted avg       0.68      0.60      0.63       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [178]:
nonlinear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.765
Precision: 0.587
Recall: 0.765
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.66       694



In [179]:
linear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.741
Precision: 0.663
Recall: 0.741
F_score: 0.681

              precision    recall  f1-score   support

           0       0.77      0.94      0.85       532
           1       0.30      0.09      0.13       162

    accuracy                           0.74       694
   macro avg       0.54      0.51      0.49       694
weighted avg       0.66      0.74      0.68       694



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [56]:
def word_vector_for_sent(list_token_embeddings, size):
      vec = np.zeros(size).reshape((1, size))
      count = 0
      for word in list_token_embeddings:
          try:
              # print(model_w2v[word].shape)
              vec += np.array(word).reshape((1, size))
              count += 1.
          except KeyError:  # handling the case where the token is not in vocabulary
              continue
      if count != 0:
          vec /= count
      return vec

In [57]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
 
 
def get_word_idx(sent: str, word: str):
    diff_vals=sent.split(" ")
    idxs=np.arange(start=0, stop=len(diff_vals), step=1)

    return idxs


def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
      Select only those subword token outputs that belong to our word of interest
      and average them."""
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # print(output.shape)
    # Only select the tokens that constitute the requested word
    # word_tokens_output = output[token_ids_word]

    return word_vector_for_sent(output,output.shape[1])


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
      that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    # print(encoded,token_ids_word)

    return get_hidden_states(encoded, token_ids_word, model, layers)


def Run(model_name='bert-base-cased',layers=None):
    # Use last four layers by default
    layers = [-4, -3, -2, -1] if layers is None else layers
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

    embedding_train=[]
    for text in train['tweet'].values:    
      idx = get_word_idx(text, '')
      # print(len(idx))
      word_embedding = get_word_vector(text, idx, tokenizer, model, layers)
      # print(word_embedding)
      embedding_train.append(word_embedding)
    embedding_test=[]
    for text in test['tweet'].values:    
      idx = get_word_idx(text, ' ')
      word_embedding = get_word_vector(text, idx, tokenizer, model, layers)
      embedding_test.append(word_embedding)
    
    return embedding_train, embedding_test

In [58]:
import gc
gc.collect()

129

In [59]:
embedding_train, embedding_test= Run("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [14]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
#     sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     return sum_embeddings / sum_mask
  
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")


In [ ]:
# embedding_train=[]
# for text in train['tweet'].values:
#   encoded_input = tokenizer(text, return_tensors='pt')
#   model_output = model(**encoded_input)
#   # print(mean_pooling(model_output, encoded_input['attention_mask']).shape)
#   embedding_train.append(mean_pooling(model_output, encoded_input['attention_mask'])[:700])
# # embedding_train=np.array(embedding_train)

In [ ]:
# embedding_test=[]
# for text in test['tweet'].values:
#   encoded_input = tokenizer(text, return_tensors='pt')
#   model_output = model(**encoded_input)
#   embedding_test.append(mean_pooling(model_output, encoded_input['attention_mask'])[:700])
# embedding_test=np.array(embedding_test)

In [64]:
embedding_train=np.array(embedding_train)
embedding_test=np.array(embedding_test)

In [65]:
embedding_train=np.squeeze(embedding_train)
embedding_test=np.squeeze(embedding_test)

In [66]:
wordvec_df_new = pd.DataFrame(embedding_train)

In [67]:
wordvec_df_test= pd.DataFrame(embedding_test)

In [68]:
gradient_boosting(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Gradient Boosting

Accuracy: 0.695
Precision: 0.667
Recall: 0.695
F_score: 0.679

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       532
           1       0.30      0.22      0.25       162

    accuracy                           0.69       694
   macro avg       0.54      0.53      0.53       694
weighted avg       0.67      0.69      0.68       694



In [69]:
Bagging_Classifier_SVM(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.690
Precision: 0.718
Recall: 0.690
F_score: 0.702

              precision    recall  f1-score   support

           0       0.82      0.76      0.79       532
           1       0.37      0.47      0.41       162

    accuracy                           0.69       694
   macro avg       0.60      0.61      0.60       694
weighted avg       0.72      0.69      0.70       694



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [70]:
Bagging_Classifier_LR(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Bagging Calssifier LR



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.667
Precision: 0.723
Recall: 0.667
F_score: 0.687

              precision    recall  f1-score   support

           0       0.83      0.71      0.77       532
           1       0.36      0.54      0.43       162

    accuracy                           0.67       694
   macro avg       0.60      0.62      0.60       694
weighted avg       0.72      0.67      0.69       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [71]:
voting_classifiers(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


   Voting Classifier



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:07:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.780
Precision: 0.747
Recall: 0.780
F_score: 0.731

              precision    recall  f1-score   support

           0       0.79      0.96      0.87       532
           1       0.59      0.18      0.27       162

    accuracy                           0.78       694
   macro avg       0.69      0.57      0.57       694
weighted avg       0.75      0.78      0.73       694



In [72]:
ensemble_stacked(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


Ensemble Stacked Classifiers



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:09:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sci

[18:12:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:14:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:16:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:18:18] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.777
Precision: 0.749
Recall: 0.777
F_score: 0.708

              precision    recall  f1-score   support

           0       0.78      0.98      0.87       532
           1       0.64      0.10      0.17       162

    accuracy                           0.78       694
   macro avg       0.71      0.54      0.52       694
weighted avg       0.75      0.78      0.71       694



In [73]:
xg_boost(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'])


             XGBoost



/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:22:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.762
Precision: 0.716
Recall: 0.762
F_score: 0.719

              precision    recall  f1-score   support

           0       0.79      0.94      0.86       532
           1       0.48      0.18      0.26       162

    accuracy                           0.76       694
   macro avg       0.63      0.56      0.56       694
weighted avg       0.72      0.76      0.72       694



In [74]:
random_forest(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Random Forest

Accuracy: 0.765
Precision: 0.587
Recall: 0.765
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.66       694



In [75]:
logistic_regression(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression

Accuracy: 0.654
Precision: 0.732
Recall: 0.654
F_score: 0.678

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       532
           1       0.36      0.60      0.45       162

    accuracy                           0.65       694
   macro avg       0.60      0.63      0.60       694
weighted avg       0.73      0.65      0.68       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [76]:
linear_svm(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM

Accuracy: 0.653
Precision: 0.718
Recall: 0.653
F_score: 0.675

              precision    recall  f1-score   support

           0       0.83      0.69      0.75       532
           1       0.35      0.54      0.42       162

    accuracy                           0.65       694
   macro avg       0.59      0.61      0.59       694
weighted avg       0.72      0.65      0.67       694



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [77]:
logistic_regression_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


 Logistic Regression



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.650
Precision: 0.714
Recall: 0.650
F_score: 0.672

              precision    recall  f1-score   support

           0       0.83      0.69      0.75       532
           1       0.34      0.53      0.41       162

    accuracy                           0.65       694
   macro avg       0.58      0.61      0.58       694
weighted avg       0.71      0.65      0.67       694



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [78]:
nonlinear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.772
Precision: 0.734
Recall: 0.772
F_score: 0.731

              precision    recall  f1-score   support

           0       0.80      0.95      0.86       532
           1       0.53      0.20      0.29       162

    accuracy                           0.77       694
   macro avg       0.66      0.57      0.58       694
weighted avg       0.73      0.77      0.73       694



In [79]:
linear_svm_grid(wordvec_df_new,train['sarcastic'],wordvec_df_test,test['sarcastic'],class_ratio='balanced')


          Linear SVM



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy: 0.739
Precision: 0.711
Recall: 0.739
F_score: 0.721

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       532
           1       0.41      0.28      0.34       162

    accuracy                           0.74       694
   macro avg       0.61      0.58      0.59       694
weighted avg       0.71      0.74      0.72       694



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
